In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import matplotlib.colors as mcolors
import healpy as hp
import math
from shapely.geometry import Point, Polygon
import plotly.graph_objects as go

In [3]:
# void EulerSearch::SetSymmetryLimits( ) {
#     // Frealign limits, original code written by Richard Henderson
#     //    DATA  ASYMTEST/' CDTOI0123456789'/
#     //    DATA  THETASTORE/90.0,  90.0,  54.7,  54.7,  31.7/
#     //    DATA  PHISTORE/360.0,  360.0, 180.0,  90.0, 180.0/
#     //    DATA  JSTORE/2,1,1,1,1/

#     wxChar symmetry_type;
#     long   symmetry_number;

#     if ( symmetry_symbol.Length( ) < 1 ) {
#         MyPrintWithDetails("Error: Must specify symmetry symbol\n");
#         DEBUG_ABORT;
#     }
#     symmetry_type = symmetry_symbol.Capitalize( )[0];
#     if ( symmetry_symbol.Length( ) == 1 ) {
#         symmetry_number = 0;
#     }
#     else {
#         if ( ! symmetry_symbol.Mid(1).ToLong(&symmetry_number) ) {
#             MyPrintWithDetails("Error: Invalid n after symmetry symbol\n");
#             DEBUG_ABORT;
#         }
#     }

#     if ( symmetry_type == 'C' ) {
#         if ( symmetry_number == 0 ) {
#             MyPrintWithDetails("Error: Invalid n after symmetry symbol\n");
#             DEBUG_ABORT;
#         }

#         phi_max     = 360.0 / symmetry_number;
#         theta_max   = 90.0;
#         test_mirror = true;

#         return;
#     }

#     if ( symmetry_type == 'D' ) {
#         if ( symmetry_number == 0 ) {
#             MyPrintWithDetails("Error: Invalid n after symmetry symbol\n");
#             DEBUG_ABORT;
#         }

#         phi_max     = 360.0 / symmetry_number;
#         theta_max   = 90.0;
#         test_mirror = false;

#         return;
#     }

#     if ( symmetry_type == 'T' ) {
#         phi_max     = 180.0;
#         theta_max   = 54.7;
#         test_mirror = false;

#         return;
#     }

#     if ( symmetry_type == 'O' ) {
#         phi_max     = 90.0;
#         theta_max   = 54.7;
#         test_mirror = false;

#         return;
#     }

#     if ( symmetry_type == 'I' ) {
#         phi_max     = 180.0;
#         theta_max   = 31.7;
#         test_mirror = false;

#         return;
#     }

#     MyPrintWithDetails("Error: Invalid symmetry symbol\n");
#     DEBUG_ABORT;
# }

In [4]:
def SetSymmetryLimits(symmetry_symbol, symmetry_number): 
    # // Frealign limits, original code written by Richard Henderson
    # //    DATA  ASYMTEST/' CDTOI0123456789'/
    # //    DATA  THETASTORE/90.0,  90.0,  54.7,  54.7,  31.7/
    # //    DATA  PHISTORE/360.0,  360.0, 180.0,  90.0, 180.0/
    # //    DATA  JSTORE/2,1,1,1,1/

    # wxChar symmetry_type;
    # long   symmetry_number;
    
    #Algorithm:
    #Check if all four points of pixel are out of the theta phi range, if so skip pixel
    #Mark pixels with 1-3 points of pixel out of range
    #Filter points in these pixels
    
    symmetry_type = symmetry_symbol
    # if ( symmetry_symbol.Length( ) < 1 ) {
    #     MyPrintWithDetails("Error: Must specify symmetry symbol\n");
    #     DEBUG_ABORT;
    # }
    # symmetry_type = symmetry_symbol.Capitalize( )[0];
    # if ( symmetry_symbol.Length( ) == 1 ) {
    #     symmetry_number = 0;
    # }
    # else {
    #     if ( ! symmetry_symbol.Mid(1).ToLong(&symmetry_number) ) {
    #         MyPrintWithDetails("Error: Invalid n after symmetry symbol\n");
    #         DEBUG_ABORT;
    #     }
    # }

    if ( symmetry_type == 'C' ):
        # if ( symmetry_number == 0 ) {
        #     MyPrintWithDetails("Error: Invalid n after symmetry symbol\n");
        #     DEBUG_ABORT;
        # }

        phi_max     = 360.0 / symmetry_number
        theta_max   = 180.0
        # test_mirror = true;

        return phi_max, theta_max
    

    if ( symmetry_type == 'D' ):
        # if ( symmetry_number == 0 ) {
        #     MyPrintWithDetails("Error: Invalid n after symmetry symbol\n");
        #     DEBUG_ABORT;
        # }

        phi_max     = 360.0 / symmetry_number
        theta_max   = 90.0
        # test_mirror = false;

        return phi_max, theta_max
    

    if ( symmetry_type == 'T' ):
        phi_max     = 180.0
        theta_max   = 54.7
        #test_mirror = false;

        return;
    

    if ( symmetry_type == 'O' ):
        phi_max     = 90.0
        theta_max   = 54.7
        # test_mirror = false;

        return phi_max, theta_max
    

    if ( symmetry_type == 'I' ):
        phi_max     = 180.0
        theta_max   = 31.7
        # test_mirror = false;

        return phi_max, theta_max
    

    # MyPrintWithDetails("Error: Invalid symmetry symbol\n");
    # DEBUG_ABORT;


In [25]:
def main(order, spacing):
    
    data = []
    #num_points = num
    # Create points for sphere
    theta = np.linspace(0, 2 * np.pi, 100)
    phi = np.linspace(0, np.pi, 50)
    theta, phi = np.meshgrid(theta, phi)
    r = 1
    # Convert to Cartesian coordinates
    x = r * np.sin(phi) * np.cos(theta)
    y = r * np.sin(phi) * np.sin(theta)
    z = r * np.cos(phi)
    
    sphere = go.Surface(x=x,y=y,z=z,colorscale='Gray',opacity=.2,showscale=False)
    data.append(sphere)
    for i in range(hp.order2npix(order)):
        points = getBoundaries(order, i)
        x = points[0] 
        y = points[1]
        z = points[2]
        scatter = go.Scatter3d(x=x,y=y,z=z,mode='markers',marker=dict(size=10,color="red",colorscale='Viridis',opacity=0.8), name="Corners")
        if(i >= 4 and i<=7):
            num_points = int (57.9*(spacing)**-0.95)
        else:
            num_points = int (64.8*(spacing)**-0.951)
        arc12, arc23, arc34, arc41 = getArcs(points, num_points)
        arcs = np.concatenate((arc12, arc23, arc34, arc41), axis = 0)
        scatters = go.Scatter3d(x=arcs[:,0],y=arcs[:,1],z=arcs[:,2],mode='markers',marker=dict(size=4,color="blue",colorscale='Viridis',opacity=0.8), name="Borders")
        
        phi_max, theta_max = SetSymmetryLimits('C', 4)
        #print(theta_max, phi_max)
        new_points, arc_lengths= getPoints(arc12, arc23, arc34, arc41, theta_max, phi_max, num_points)
        if(np.shape(new_points)[0] == 0):
            continue
        x1 = new_points[:,0]
        y1 = new_points[:,1]
        z1 = new_points[:,2]
        # Create scatter3d plot
        scatter1 = go.Scatter3d(x=x1,y=y1,z=z1,mode='markers',marker=dict(size=2,colorscale='Viridis',opacity=1), name="Pixel " + str(i))

        
        data.append(scatter)
        data.append(scatters)
        data.append(scatter1)
        
        
    
    fig = go.Figure(data=data)
    fig.update_layout(
        title=f'HEALPix Points on Sphere (order={order})',
        scene=dict(
            xaxis=dict(showbackground=False, visible=False),
            yaxis=dict(showbackground=False, visible=False),
            zaxis=dict(showbackground=False, visible=False),
            aspectmode='data'
        ),
        width=800,
        height=800
    )
    # Display the plot
    fig.show()
    #return arc_lengths, small_arcs

# t,s  = main(0,0)

In [27]:
main(0,2.5)

In [7]:
def getBoundaries(order, i):
    x_coor = []
    y_coor = []
    z_coor = []
    # for i in range(hp.order2npix(order)):
    test = hp.boundaries(hp.order2nside(order), i, step=1)
    x_coor.extend(test[0])
    y_coor.extend(test[1])
    z_coor.extend(test[2])
    
    return x_coor, y_coor, z_coor

In [8]:
def getArcs(points, num_points):
    
    p1, p2, p3, p4= np.transpose(points)
    arc12 = great_circle_arc(p1, p2, num_points)
    arc23 = great_circle_arc(p2, p3, num_points)
    arc34 = great_circle_arc(p3, p4, num_points)
    arc41 = great_circle_arc(p4, p1, num_points)
    
    return arc12, arc23, arc34, arc41
    

In [9]:
def getPoints(arc12, arc23, arc34, arc41,theta_max, phi_max, num_points = 50):
    new_points = []
    all_arc_lengths = []
    thetas = []
    phis = []
    
    #This function returns arcs both from top to bottom as well as side to side
    for i in range(num_points):
        arc = great_circle_arc(arc12[i], arc34[num_points - 1 - i], num_points)
        for j in range(num_points):
            theta, phi = spherical(arc[j])
            thetas.append(theta)
            phis.append(phi)
            if(theta < theta_max and phi<phi_max):
                new_points.append(arc[j])
                
    # print("theta:")
    # print(np.max((thetas)) *180/np.pi)
    # print("phi")
    # print(np.max((phis)) *180/np.pi)
        # arc_lengths=[]
        # for j in range(num_points-1):
        #     arc_lengths.append(angle_finder(arc[j], arc[j+1]))
        # all_arc_lengths.append(arc_lengths)
    # for i in range(num_points):
    #     arc = great_circle_arc(arc23[i], arc41[num_points -1 - i], num_points)
    #     new_points.extend(arc)
    #     arc_lengths=[]
    #     for j in range(num_points-1):
    #         arc_lengths.append(angle_finder(arc[j], arc[j+1]))
    #     all_arc_lengths.append(arc_lengths)


    

                
                    
    return np.array(new_points), np.array(all_arc_lengths)

In [10]:
def great_circle_arc(p1, p2, num_points=50):
    """Generates points along a great circle arc between two points."""
    p1_norm = p1 / np.linalg.norm(p1)
    p2_norm = p2 / np.linalg.norm(p2)
    points = []
    for i in range(num_points):
        t = i / (num_points - 1)
        # Linear interpolation and normalization
        pt = (1 - t) * p1_norm + t * p2_norm
        pt_norm = pt / np.linalg.norm(pt)
        points.append(pt_norm)
    return np.array(points)

In [11]:
def diff_in_angle(p1, p2):
   new_p1 = spherical(p1)
   new_p2 = spherical(p2)
   theta= abs(math.degrees(new_p1[0]) - math.degrees(new_p2[0]))
   phi = abs(math.degrees(new_p1[1]) - math.degrees(new_p2[1]))
   return theta, phi

In [12]:
def spherical(point):
    return math.degrees(np.arccos(point[2])),  math.degrees(math.atan2(point[1], point[0])) % 360

In [13]:
test_cases = [
    (1, 0, 0),  # Along +x axis
    (0, 1, 0),  # Along +y axis
    (0, 0, 1),  # Along +z axis
    (-1, -1, 0),  # In third quadrant of xy-plane
    (1, 1, 1),  # General case
    (0, -1, 0),  # Along -y axis
    (-1, 0, 0),  # Along -x axis
    (0, 0, -1),  # Along -z axis
]

for i in range(8):
    theta, phi = spherical(test_cases[i])
    print(f"Theta: {theta:.2f}°, Phi: {phi:.2f}°")

Theta: 90.00°, Phi: 0.00°
Theta: 90.00°, Phi: 90.00°
Theta: 0.00°, Phi: 0.00°
Theta: 90.00°, Phi: 225.00°
Theta: 0.00°, Phi: 45.00°
Theta: 90.00°, Phi: 270.00°
Theta: 90.00°, Phi: 180.00°
Theta: 180.00°, Phi: 0.00°


In [14]:
import math

def spherical_angle_diff(point1, point2):
    """
    Calculates the difference in theta and phi angles between two points on a sphere given their Cartesian coordinates.

    Args:
        point1: A tuple of length 3 representing the Cartesian coordinates of the first point.
        point2: A tuple of length 3 representing the Cartesian coordinates of the second point.

    Returns:
        A tuple containing the difference in theta and phi angles.
    """
    
    def to_spherical(point):
        x, y, z = point
        r = math.sqrt(x**2 + y**2 + z**2)
        theta = math.atan2(y, x)
        phi = math.acos(z / r)
        return theta, phi

    theta1, phi1 = to_spherical(point1)
    theta2, phi2 = to_spherical(point2)

    return (theta2 - theta1) *180/np.pi, (phi2 - phi1)*180/np.pi

In [15]:
def angle_finder(point1, point2):

    point1 = np.array(point1)
    point2 = np.array(point2)

    # Calculate the dot product of the two vectors
    dot_product = np.dot(point1, point2)

     # Ensure the value is within the valid range for arccos (-1 to 1)
    cos_theta = np.clip(dot_product, -1, 1)
    
    # Calculate the angle in radians using arccos
    angle = math.degrees(math.acos(cos_theta))

    return angle

In [16]:
    
    def to_spherical(point):
        x, y, z = point
        r = math.sqrt(x**2 + y**2 + z**2)
        theta = math.atan2(y, x)
        phi = math.acos(z / r)
        return theta, phi
